In [10]:
import pandas as pd
import os
import numpy as np

In [ ]:
# read in data
df = pd.read_feather("../Data/hospital.feather")
df.head()

In [3]:
# show columns
df.columns

Index(['index', 'Health.Service.Area', 'Hospital.County',
       'Operating.Certificate.Number', 'Facility.ID', 'Facility.Name',
       'Age.Group', 'Zip.Code...3.digits', 'Gender', 'Race', 'Ethnicity',
       'Length.of.Stay', 'Type.of.Admission', 'Patient.Disposition',
       'Discharge.Year', 'CCS.Diagnosis.Code', 'CCS.Diagnosis.Description',
       'CCS.Procedure.Code', 'CCS.Procedure.Description', 'APR.DRG.Code',
       'APR.DRG.Description', 'APR.MDC.Code', 'APR.MDC.Description',
       'APR.Severity.of.Illness.Code', 'APR.Severity.of.Illness.Description',
       'APR.Risk.of.Mortality', 'APR.Medical.Surgical.Description',
       'Source.of.Payment.1', 'Source.of.Payment.2', 'Source.of.Payment.3',
       'Attending.Provider.License.Number',
       'Operating.Provider.License.Number', 'Other.Provider.License.Number',
       'Birth.Weight', 'Abortion.Edit.Indicator',
       'Emergency.Department.Indicator', 'Total.Charges', 'Total.Costs'],
      dtype='object')

In [73]:
# null columns and abortion correlation

print(f'total dataset: {len(df.index)} rows')
print(f'total null: {df["Facility.ID"].isna().sum()} empty facility ID rows, {df["Facility.ID"].isna().sum()/len(df.index)*100:.3f}% of dataset')
display(df[df["Facility.ID"].isna()]["CCS.Diagnosis.Description"].value_counts(normalize=True)[0:10])
display(df[df["Facility.ID"].isna()]["CCS.Diagnosis.Description"].value_counts()[0:10])

print(f'\nAbortions where facility ID is null:')
display(df[df["Facility.ID"].isna()]["Abortion.Edit.Indicator"].value_counts())

print(f'\nAverage abortion total cost ${df[df["Abortion.Edit.Indicator"] == "Y"]["Total.Charges"].mean():.2f}:')
print(f'Average abortion patient payment ${df[df["Abortion.Edit.Indicator"] == "Y"]["Total.Costs"].mean():.2f}:')

total dataset: 2622133 rows
total null: 4887 empty facility ID rows, 0.186% of dataset


SPONTANEOUS ABORTION      0.426028
OTHR PREGNANCY COMPL      0.339063
INDUCED ABORTION          0.163700
OTH COMP BIRTH/PUERPRM    0.011050
ECTOPIC PREGNANCY         0.006957
POLYHYDRAMNIOS ET AL      0.004502
POSTABORTION COMPL        0.003888
PREGNANCY HEMORRHAG       0.003479
EARLY LABOR               0.003479
MOOD DISORDERS            0.003069
Name: CCS.Diagnosis.Description, dtype: float64

SPONTANEOUS ABORTION      2082
OTHR PREGNANCY COMPL      1657
INDUCED ABORTION           800
OTH COMP BIRTH/PUERPRM      54
ECTOPIC PREGNANCY           34
POLYHYDRAMNIOS ET AL        22
POSTABORTION COMPL          19
PREGNANCY HEMORRHAG         17
EARLY LABOR                 17
MOOD DISORDERS              15
Name: CCS.Diagnosis.Description, dtype: int64


Abortions where facility ID is null:


Y    4887
Name: Abortion.Edit.Indicator, dtype: int64


Average abortion total cost $11983.26:
Average abortion patient payment $5532.49:


In [56]:
# data types

display(df.dtypes)

display(df.dtypes.value_counts())

index                                    int32
Health.Service.Area                     object
Hospital.County                         object
Operating.Certificate.Number           float64
Facility.ID                            float64
Facility.Name                           object
Age.Group                               object
Zip.Code...3.digits                     object
Gender                                  object
Race                                    object
Ethnicity                               object
Length.of.Stay                          object
Type.of.Admission                       object
Patient.Disposition                     object
Discharge.Year                           int32
CCS.Diagnosis.Code                     float64
CCS.Diagnosis.Description               object
CCS.Procedure.Code                     float64
CCS.Procedure.Description               object
APR.DRG.Code                             int32
APR.DRG.Description                     object
APR.MDC.Code 

object     23
float64     9
int32       6
dtype: int64

In [116]:
for column in df.columns:
    numunique = len(df[column].unique())
    if numunique > 5:
        print(f"number of items in [{column}]: {numunique}")
        

number of items in [index]: 2622133
number of items in [Health.Service.Area]: 9
number of items in [Hospital.County]: 58
number of items in [Operating.Certificate.Number]: 190
number of items in [Facility.ID]: 226
number of items in [Facility.Name]: 225
number of items in [Zip.Code...3.digits]: 76
number of items in [Length.of.Stay]: 120
number of items in [Type.of.Admission]: 6
number of items in [Patient.Disposition]: 20
number of items in [CCS.Diagnosis.Code]: 263
number of items in [CCS.Diagnosis.Description]: 263
number of items in [CCS.Procedure.Code]: 233
number of items in [CCS.Procedure.Description]: 233
number of items in [APR.DRG.Code]: 316
number of items in [APR.DRG.Description]: 316
number of items in [APR.MDC.Code]: 26
number of items in [APR.MDC.Description]: 26
number of items in [Source.of.Payment.1]: 10
number of items in [Source.of.Payment.2]: 11
number of items in [Source.of.Payment.3]: 11
number of items in [Attending.Provider.License.Number]: 30087
number of item

In [113]:
# some more data preprocessing

# bin length of stay into categorical

df["Length.of.Stay"].unique()
def mapper(staylength):
    if staylength != "120 +":
        if int(staylength) <= 30:
            return "<30"
        elif int(staylength) <= 60:
            return "30-60"
        elif int(staylength) <= 90:
            return "60-90"
        elif int(staylength) <= 120:
            return "90-120"
    else:
        return staylength
df["Length.of.Stay.Binned"] = [mapper(stay) for stay in df["Length.of.Stay"]]


# check if it processed correctly
pd.crosstab(index=df["Length.of.Stay"], columns=df["Length.of.Stay.Binned"])

Length.of.Stay.Binned,120 +,30-60,60-90,90-120,<30
Length.of.Stay,,,,,
1,0,0,0,0,428644
10,0,0,0,0,44466
100,0,0,0,88,0
101,0,0,0,67,0
102,0,0,0,71,0
...,...,...,...,...,...
95,0,0,0,81,0
96,0,0,0,89,0
97,0,0,0,94,0


In [109]:
# some more data preprocessing

# POC indicator
df["Race"].unique()
df["POC"] = [1 if race != "White" else 0 for race in df["Race"]]

# check if it processed correctly
pd.crosstab(index=df["Race"], columns=df["POC"])

POC,0,1
Race,,
Black/African American,0,498561
Other Race,0,493854
Unknown,0,28340
White,1601378,0


In [ ]:
# save file
# df.to_feather("../Data/hospital.feather")

In [75]:

factorized = pd.read_feather("../Data/hospital-factorized.feather")
factorized.head()

,Health.Service.Area,Hospital.County,Operating.Certificate.Number,Facility.ID,Age.Group,Zip.Code...3.digits,Gender,Race,Ethnicity,Length.of.Stay,...,Source.of.Payment.3,Attending.Provider.License.Number,Operating.Provider.License.Number,Other.Provider.License.Number,Birth.Weight,Abortion.Edit.Indicator,Emergency.Department.Indicator,Total.Charges,Total.Costs,Extended.Stay
0,Capital/Adiron,Albany,101000,1,18 to 29,NaN,F,White,Not Span/Hispanic,1,...,None,3623,NaN,NaN,0,N,Y,4476.23,1672.65,0
1,Capital/Adiron,Albany,101000,1,50 to 69,NaN,M,White,Not Span/Hispanic,15,...,Blue Cross,216951,216951,NaN,0,N,Y,148612.34,51414.70,0
2,Capital/Adiron,Albany,101000,1,70 or Older,NaN,M,White,Not Span/Hispanic,3,...,Self-Pay,3076,NaN,NaN,0,N,N,16561.99,4032.49,0
3,Capital/Adiron,Albany,101000,1,70 or Older,NaN,F,White,Not Span/Hispanic,5,...,Self-Pay,140796,140796,170254,0,N,N,13593.51,4947.81,0
4,Capital/Adiron,Albany,101000,1,70 or Older,NaN,F,White,Not Span/Hispanic,4,...,Self-Pay,170254,170254,170254,0,N,N,31962.58,16988.33,0


In [104]:
# try to do some categorical variables correlation

from scipy.stats import chi2_contingency
crosstab = pd.crosstab(index=df["Age.Group"], columns=df['Length.of.Stay.Binned'], normalize=True)
print(crosstab)
result = chi2_contingency(crosstab)

result

Length.of.Stay.Binned     120 +     30-60     60-90    90-120       <30
Age.Group                                                              
0 to 17                0.000137  0.001413  0.000456  0.000201  0.145517
18 to 29               0.000087  0.000998  0.000224  0.000074  0.106155
30 to 49               0.000198  0.002270  0.000479  0.000164  0.205645
50 to 69               0.000293  0.004032  0.000726  0.000249  0.254094
70 or Older            0.000131  0.003628  0.000508  0.000152  0.272171


(0.0006924414784367294,
 1.0,
 16,
 array([[1.24787540e-04, 1.82296849e-03, 3.53405073e-04, 1.24167827e-04,
         1.45299062e-01],
        [9.08405017e-05, 1.32705054e-03, 2.57265222e-04, 9.03893750e-05,
         1.05772096e-01],
        [1.76341936e-04, 2.57610490e-03, 4.99409915e-04, 1.75466197e-04,
         2.05327534e-01],
        [2.19118586e-04, 3.20101093e-03, 6.20555707e-04, 2.18030412e-04,
         2.55135449e-01],
        [2.33643569e-04, 3.41320028e-03, 6.61691246e-04, 2.32483263e-04,
         2.72047927e-01]]))